In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import pprint
import os
import datetime



In [2]:
winemag2 = "./csv/winemag-data-130k-rob.csv"

winemag_data2 = pd.read_csv(winemag2)

winemag_data2.count()

Unnamed: 0               129971
country                  129908
description              129971
designation               92506
points                   129971
price                    120975
province                 129908
region_1                 108724
region_2                  50511
taster_name              103727
taster_twitter_handle     98758
title                    129971
variety                  129970
winery                   129971
dtype: int64

In [3]:
#strip out america as a province so we can compare states
winemag_data2.head()
#df = df[df.line_race != 0]
winemag_data2 = winemag_data2[winemag_data2.province != 'America']

In [4]:
#Remove unnecessary/unused columns
winemag2_df = winemag_data2.drop(columns=["Unnamed: 0","description","region_1","region_2","taster_name","taster_twitter_handle","title"])
winemag2_df.head()

,country,designation,points,price,province,variety,winery
0,Italy,Vulkà Bianco,87,NaN,Sicily & Sardinia,White Blend,Nicosia
1,Portugal,Avidagos,87,15.0,Douro,Portuguese Red,Quinta dos Avidagos
2,US,NaN,87,14.0,Oregon,Pinot Gris,Rainstorm
3,US,Reserve Late Harvest,87,13.0,Michigan,Riesling,St. Julian
4,US,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Pinot Noir,Sweet Cheeks


In [5]:
#check row counts
winemag_all_rows = winemag2_df
winemag_all_rows.count()

country        129813
designation     92437
points         129876
price          120880
province       129813
variety        129875
winery         129876
dtype: int64

In [6]:
# get rid of the indexing from xls
winemag_all_rows=winemag_all_rows.reset_index(drop=True)
winemag_all_rows.head()

,country,designation,points,price,province,variety,winery
0,Italy,Vulkà Bianco,87,NaN,Sicily & Sardinia,White Blend,Nicosia
1,Portugal,Avidagos,87,15.0,Douro,Portuguese Red,Quinta dos Avidagos
2,US,NaN,87,14.0,Oregon,Pinot Gris,Rainstorm
3,US,Reserve Late Harvest,87,13.0,Michigan,Riesling,St. Julian
4,US,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Pinot Noir,Sweet Cheeks


In [7]:
# have a look at the country counts and averages
by_country_count = winemag_all_rows.groupby("country").count()
by_country_count
by_country_mean = winemag_all_rows.groupby("country").mean().reset_index() #get country to show
by_country_mean


,country,points,price
0,Argentina,86.710263,24.510117
1,Armenia,87.500000,14.500000
2,Australia,88.580507,35.437663
3,Austria,90.101345,30.762772
4,Bosnia and Herzegovina,86.500000,12.500000
5,Brazil,84.673077,23.765957
6,Bulgaria,87.936170,14.645390
7,Canada,89.369650,35.712598
8,Chile,86.493515,20.786458
9,China,89.000000,18.000000


In [8]:
# just in case I want to look at the data 
winemag_all_rows.to_csv("./csv/winemag_cleaned.csv")


In [10]:
# verify data
winemag_all_rows.shape

(129876, 7)

In [11]:
# clean up blank entries
winemag_all_cleaned = winemag_all_rows.dropna()

In [12]:
# Strip out US specific data
winemag_US = winemag_all_rows.loc[winemag_all_rows['country'] == "US"]
winemag_US.head()

,country,designation,points,price,province,variety,winery
2,US,NaN,87,14.0,Oregon,Pinot Gris,Rainstorm
3,US,Reserve Late Harvest,87,13.0,Michigan,Riesling,St. Julian
4,US,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Pinot Noir,Sweet Cheeks
10,US,Mountain Cuvée,87,19.0,California,Cabernet Sauvignon,Kirkland Signature
12,US,NaN,87,34.0,California,Cabernet Sauvignon,Louis M. Martini


In [13]:
# order by state & use as index
by_US_state = winemag_US.groupby("province").mean().reset_index()
by_US_state.head()

,province,points,price
0,Arizona,84.926829,29.200000
1,California,88.627776,39.041048
2,Colorado,86.117647,32.985294
3,Connecticut,84.000000,15.333333
4,Hawaii,87.000000,28.000000


In [14]:
# adjust the by state values for better viewing
by_state_mean = by_US_state.sort_values(by=['points'], ascending=False)
by_state_mean["points"] = by_state_mean["points"] - 80

In [ ]:
# adjust country values to better highlight the differences
by_country_mean = by_country_mean.sort_values(by=['points'], ascending=False)
by_country_mean["points"] = by_country_mean["points"] - 80